# Data-Driven Astronomy #2 

In [2]:
#imports 
import numpy as np

## Converting to decimal degrees

In [1]:
def hms2dec(h, m, s):
    return 15*(h + m/60 + s/3600)

def dms2dec(d, m, s):
    sign = d/abs(d)
    return sign*(abs(d) + m/60 + s/3600)

## Angular distance

In [ ]:
def angular_dist(RA1, dec1, RA2, dec2):
    
    # Convert to radians
    r1 = np.radians(RA1)
    d1 = np.radians(dec1)
    r2 = np.radians(RA2)
    d2 = np.radians(dec2)

    a = np.sin(np.abs(d1 - d2)/2)**2
    b = np.cos(d1)*np.cos(d2)*np.sin(np.abs(r1 - r2)/2)**2

    angle = 2*np.arcsin(np.sqrt(a + b))
    
    # Convert back to degrees
    return np.degrees(angle)

## Loading catalogues

In [3]:
def import_bss():
    res = []
    data = np.loadtxt('bss.dat', usecols=range(1, 7))
    for i, row in enumerate(data, 1):
        res.append((i, hms2dec(row[0], row[1], row[2]), dms2dec(row[3], row[4], row[5])))
    return res

def import_super():
    data = np.loadtxt('super.csv', delimiter=',', skiprows=1, usecols=(0, 1))
    res = []
    for i, row in enumerate(data, 1):
        res.append((i, row[0], row[1]))
    return res

## Full cross-match

In [4]:
def crossmatch(cat1, cat2, max_radius):
    matches = []
    no_matches = []
    for id1, ra1, dec1 in cat1:
        closest_dist = np.inf
        closest_id2 = None
        for id2, ra2, dec2 in cat2:
            dist = angular_dist(ra1, dec1, ra2, dec2)
            if dist < closest_dist:
                closest_id2 = id2
                closest_dist = dist
        
        # Ignore match if it's outside the maximum radius
        if closest_dist > max_radius:
            no_matches.append(id1)
        else:
            matches.append((id1, closest_id2, closest_dist))

    return matches, no_matches

## Creating a random catalogue

In [5]:
def create_cat(n):
    ras = np.random.uniform(0, 360, size=(n, 1))
    decs = np.random.uniform(-90, 90, size=(n, 1))
    return np.hstack((ras, decs))